In [ ]:
from langchain_community.document_loaders import PDFMinerLoader
import pandas as pd
import os
import re
import sys
sys.path.append('d:/Local-RAG-Assistant-Chatbot/') # change to your own path
from datasets import load_dataset

In [2]:
dataset = load_dataset('squad', split='validation')
data_dict = {
    'id': [row['id'] for row in dataset],
    'title': [row['title'] for row in dataset],
    'context': [row['context'] for row in dataset],
    'question': [row['question'] for row in dataset],
    'answers': [row['answers'] for row in dataset]
}

df = pd.DataFrame(data_dict)
df = df[df['title'] == 'Computational_complexity_theory']

In [4]:
df['first_answer'] = df['answers'].apply(lambda x: x['text'][1])

In [5]:
df = df.drop_duplicates(subset='context', keep='first')

In [7]:
df.shape

(48, 6)

In [8]:
from datasets import Dataset
# Function to evaluate as Llama index does not support async evaluation for HFInference API
def generate_responses(test_questions, pdf_file, test_answers):
  outputs = [chat_bot.generate_response(q, pdf_file) for q in test_questions]

  answers = []
  contexts = []
  for output in outputs:
    answers.append(output['answer'])
    contexts.append([doc.page_content for doc in output['source_documents']])
  dataset_dict = {
        "question": test_questions,
        "answer": answers,
        "contexts": contexts,
  }
  if test_answers is not None:
    dataset_dict["ground_truth"] = test_answers
  #ds = Dataset.from_dict(dataset_dict)
  return dataset_dict

In [ ]:
import json
from src import chatbot

import time
path =  "D:/Local-RAG-Assistant-Chatbot/config.yaml" # change to your own path
pdf_path = "D:/Local-RAG-Assistant-Chatbot/eval/merged_pdfs/squad_comput_theory.pdf" # change to your own path
chat_bot = chatbot.PDFChatBot(config_path=path)

# begin

questions = df.loc[:]['question'].values.tolist()
test_answers = df.loc[:]['first_answer'].values.tolist()
dataset_dict = generate_responses(questions, pdf_path, test_answers)
output_path = "./outputs/"
if not os.path.exists(output_path):
    os.makedirs(output_path)
output_file_path = output_path + f"outputs_eval_squad.json"
with open(output_file_path, 'w') as json_file:
    json.dump(dataset_dict, json_file, indent=4)


In [ ]:
# Split the large SQuAD outputs into 4 chunks to avoid the openAI rate limit
def split_json_data(input_data, chunk_size):
    num_keys = 4  # Assuming 4 keys in the original JSON structure
    data_chunks = []

    for i in range(num_keys):
        key_chunk = {}
        for key, values in input_data.items():
            # Split values for the current key into chunks of specified size
            start_idx = i * chunk_size
            end_idx = (i + 1) * chunk_size
            key_chunk[key] = values[start_idx:end_idx]
        data_chunks.append(key_chunk)
    
    return data_chunks

# Load the input JSON file
input_file_path = output_file_path  # Path to your input JSON file
output_file_prefix = 'outputs_eval_squad_'  # Prefix for output file names
chunk_size = 12  # Number of values per key in each chunk

with open(input_file_path, 'r') as input_file:
    input_data = json.load(input_file)
    
    # Split the input data into chunks
    data_chunks = split_json_data(input_data, chunk_size=chunk_size)
    
    # Create and write output JSON files
    for i, chunk in enumerate(data_chunks):
        output_file_path = f'{output_file_prefix}{i + 1}.json'
        
        with open(output_file_path, 'w') as output_file:
            json.dump(chunk, output_file, indent=4)

        print(f'Successfully wrote {output_file_path}')

print('Splitting into JSON files completed.')